## Preprocessing version 5
### Details
this version 
- uses all data
- has validation split
- uses bert tokenizer
- does not train embedding

In [1]:
doc_name = "../pointer-generator/data2/val.txt.src"
summ_name = "../pointer-generator/data2/val.txt.tgt.tagged"
out_dir = "data-cnn/"
vocab_name = out_dir+"vocab.json"
data_seq_name = out_dir+"valid_seq.json"#out_dir+"train_seq.json"
valid_seq_name = out_dir+"valid_seq0.json"
task_name = "cnn"
num_threads = 4
corpus_size = 88888888
validation_split = 0.

In [2]:
import os
import csv
import json
import numpy as np
from pytorch_pretrained_bert.tokenization import BertTokenizer
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool
from gensim.models.callbacks import CallbackAny2Vec

In [3]:
# !pip install --user pytorch_pretrained_bert
# !pip install --user gensim

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)
# "[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]"
UNK = "[UNK]"
BOS = "[CLS]"
EOS = "[SEP]"
PAD = "[PAD]"

In [5]:
if task_name == 'giga':
    token_mappings = {'UNK':UNK, '-lrb-':'(', '-rrb-':')'}
else:
    token_mappings = {'<unk>':UNK, '<t>':'', '</t>':''}

In [6]:
def getlines(name):
    total = !wc -l {name}
    return int(total[0].split()[0])

In [7]:
summaries = []
articles = []

with open(doc_name, newline='', encoding='utf-8') as f:
    total = getlines(doc_name)
    for i,line in tqdm(enumerate(f), total=total):
        line = line.strip()
        articles.append(line)
with open(summ_name, newline='', encoding='utf-8') as f:
    total = getlines(summ_name)
    for i,line in tqdm(enumerate(f), total=total):
        line = line.strip()
        summaries.append(line)

In [8]:
summaries = summaries[:min(corpus_size, len(summaries))]
articles = articles[:min(corpus_size, len(summaries))]

In [9]:
def task(s):
    for t, t_p in token_mappings.items():
        s = s.replace(t, t_p)
    ts = tokenizer.tokenize(s)
    return [BOS] + ts + [EOS]

In [10]:
with Pool(num_threads) as p:
    chunksize = 200
    summaries = list(tqdm(p.imap(task, summaries, chunksize=chunksize), total=len(summaries)))
    articles = list(tqdm(p.imap(task, articles, chunksize=chunksize), total=len(articles)))

In [12]:
num_summ = len(summaries)
index = 88
print("[summary]", summaries[index])
print("[documen]", articles[index])

[summary] ['[CLS]', 'campaign', 'to', 'support', 'tunisian', 'tourism', 'takes', 'hold', 'on', 'social', 'media', '.', 'tourism', 'official', ':', 'one', 'in', 'five', 'tunisian', '##s', 'depends', 'on', 'tourists', 'to', 'make', 'a', 'living', '.', '#', 'je', '##su', '##is', '##bard', '##o', 'and', '#', 'pray', '##fort', '##uni', '##sia', 'are', 'trend', '##ing', '.', '[SEP]']
[documen] ['[CLS]', '-', 'l', '##rb', '-', 'cnn', '-', 'rr', '##b', '-', '`', '`', 'i', 'will', 'come', 'to', 'tunisia', '.', "'", "'", 'it', "'", 's', 'a', 'bold', 'pledge', ',', 'some', 'might', 'say', '.', 'but', 'people', 'across', 'the', 'globe', 'are', 'making', 'that', 'vow', 'despite', 'the', 'deadly', 'attack', 'in', 'the', 'tunis', 'on', 'wednesday', '.', 'twenty', '-', 'three', 'people', ',', 'most', 'of', 'them', 'tourists', ',', 'were', 'killed', 'after', 'gunmen', 'opened', 'fire', 'inside', 'the', 'bard', '##o', 'museum', ',', 'a', 'stone', "'", 's', 'throw', 'from', 'the', 'parliament', 'building

In [13]:
print(len(tokenizer.vocab))
print(type(tokenizer.vocab))

30522
<class 'collections.OrderedDict'>


In [14]:
# print(tokenizer.basic_tokenizer.never_split)
tokens = tokenizer.tokenize(BOS+" I thought U would not split this. "+EOS)
idx = tokenizer.convert_tokens_to_ids(tokens)
print(tokens)
print(idx)

['[CLS]', 'i', 'thought', 'u', 'would', 'not', 'split', 'this', '.', '[SEP]']
[101, 1045, 2245, 1057, 2052, 2025, 3975, 2023, 1012, 102]


In [15]:
json.dump(tokenizer.vocab, open(vocab_name, "w"))

In [16]:
def task2(s):
    idxs = tokenizer.convert_tokens_to_ids(s)
    return idxs #np.asarray(idxs, dtype=np.int64)

In [17]:
seqdata = {'summary':[], 'document':[]}
valseqdata = {'summary':[], 'document':[]}
with Pool(num_threads) as p:
    chunksize = 200
    summ_seqs = list(tqdm(p.imap(task2, summaries, chunksize=chunksize), total=num_summ))
    docu_seqs = list(tqdm(p.imap(task2, articles, chunksize=chunksize), total=num_summ))

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (773 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1873 > 512). Running this sequence through BERT will result in indexing errors


Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1675 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (876 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (582 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (646 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1081 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1845 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (898 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (599 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1065 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1089 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence lengt

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (875 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (2051 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1073 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (736 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (572 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (2081 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1184 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (821 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1481 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (537 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence lengt

...

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (669 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1307 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1159 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (792 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1232 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence lengt

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (665 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (943 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (627 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (976 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (707 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length f

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1947 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1579 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (791 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (2145 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (2059 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence leng

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (523 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1655 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1203 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1550 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (779 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence lengt

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (582 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (886 > 512). Running this sequence through BERT will result in indexing errors


In [18]:
val_set = np.random.randint(0, num_summ, size=int(validation_split*num_summ))
for i in range(num_summ):
    if i in val_set:
        valseqdata['summary'].append(summ_seqs[i])
        valseqdata['document'].append(docu_seqs[i])
    else:
        seqdata['summary'].append(summ_seqs[i])
        seqdata['document'].append(docu_seqs[i])

In [19]:
json.dump(seqdata, open(data_seq_name, "w"))
json.dump(valseqdata, open(valid_seq_name, "w"))

TypeError: Object of type ndarray is not JSON serializable

In [ ]:
lengths = []
key = 'document'
for doc in seqdata[key]:
    lens = len(doc)
    lengths.append(lens)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(lengths, 50)
plt.title(key +' length')
plt.show()